In [1]:
import json
import pandas as pd
import numpy as np

import re

from sqlalchemy import create_engine
import psycopg2

import matplotlib.pyplot as plt

from datetime import datetime

In [2]:
# Load and examine the dataset
df = pd.read_csv('/Users/rotation/Desktop/Dataclass/capstone-project/archive_boston/crime.csv', encoding='latin-1')
df.head()

,INCIDENT_NUMBER,OFFENSE_CODE,OFFENSE_CODE_GROUP,OFFENSE_DESCRIPTION,DISTRICT,REPORTING_AREA,SHOOTING,OCCURRED_ON_DATE,YEAR,MONTH,DAY_OF_WEEK,HOUR,UCR_PART,STREET,Lat,Long,Location
0,I182070945,619,Larceny,LARCENY ALL OTHERS,D14,808,NaN,2018-09-02 13:00:00,2018,9,Sunday,13,Part One,LINCOLN ST,42.357791,-71.139371,"(42.35779134, -71.13937053)"
1,I182070943,1402,Vandalism,VANDALISM,C11,347,NaN,2018-08-21 00:00:00,2018,8,Tuesday,0,Part Two,HECLA ST,42.306821,-71.060300,"(42.30682138, -71.06030035)"
2,I182070941,3410,Towed,TOWED MOTOR VEHICLE,D4,151,NaN,2018-09-03 19:27:00,2018,9,Monday,19,Part Three,CAZENOVE ST,42.346589,-71.072429,"(42.34658879, -71.07242943)"
3,I182070940,3114,Investigate Property,INVESTIGATE PROPERTY,D4,272,NaN,2018-09-03 21:16:00,2018,9,Monday,21,Part Three,NEWCOMB ST,42.334182,-71.078664,"(42.33418175, -71.07866441)"
4,I182070938,3114,Investigate Property,INVESTIGATE PROPERTY,B3,421,NaN,2018-09-03 21:05:00,2018,9,Monday,21,Part Three,DELHI ST,42.275365,-71.090361,"(42.27536542, -71.09036101)"


In [3]:
# Examine available columns
df.columns.tolist()

['INCIDENT_NUMBER',
 'OFFENSE_CODE',
 'OFFENSE_CODE_GROUP',
 'OFFENSE_DESCRIPTION',
 'DISTRICT',
 'REPORTING_AREA',
 'SHOOTING',
 'OCCURRED_ON_DATE',
 'YEAR',
 'MONTH',
 'DAY_OF_WEEK',
 'HOUR',
 'UCR_PART',
 'STREET',
 'Lat',
 'Long',
 'Location']

In [4]:
df.apply(pd.Series.nunique)


INCIDENT_NUMBER        282517
OFFENSE_CODE              222
OFFENSE_CODE_GROUP         67
OFFENSE_DESCRIPTION       244
DISTRICT                   12
REPORTING_AREA            879
SHOOTING                    1
OCCURRED_ON_DATE       233229
YEAR                        4
MONTH                      12
DAY_OF_WEEK                 7
HOUR                       24
UCR_PART                    4
STREET                   4657
Lat                     18178
Long                    18178
Location                18194
dtype: int64

In [5]:
df.drop_duplicates(subset="INCIDENT_NUMBER", inplace=True)
df.drop_duplicates(subset="OFFENSE_CODE", inplace=True)
df.drop_duplicates(subset="OFFENSE_CODE", inplace=True)
df.drop_duplicates(subset="OFFENSE_DESCRIPTION", inplace=True)

In [6]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
OFFENSE_CODE,202.0,2052.787129,1082.227728,111.000000,1102.750000,2008.500000,3007.750000,3831.000000
YEAR,202.0,2017.782178,0.655690,2015.000000,2018.000000,2018.000000,2018.000000,2018.000000
MONTH,202.0,8.113861,1.642685,1.000000,8.000000,8.000000,9.000000,12.000000
HOUR,202.0,13.455446,6.226288,0.000000,11.000000,14.000000,18.000000,23.000000
Lat,199.0,41.890192,4.332555,-1.000000,42.302025,42.331521,42.350142,42.378773
Long,199.0,-70.378957,7.008211,-71.169499,-71.093959,-71.078664,-71.062154,-1.000000


In [7]:
df.groupby(['MONTH','DISTRICT']).count()


INCIDENT_NUMBER  OFFENSE_CODE  OFFENSE_CODE_GROUP  \
MONTH DISTRICT                                                      
1     E13                     1             1                   1   
      E18                     1             1                   1   
2     B2                      1             1                   1   
3     B2                      1             1                   1   
      D14                     1             1                   1   
      D4                      1             1                   1   
4     B2                      1             1                   1   
      D14                     1             1                   1   
      E5                      1             1                   1   
5     A1                      1             1                   1   
      B3                      1             1                   1   
      D14                     1             1                   1   
      E13                     1             1                   1   
      E18                     1             1                   1   
6     A1                      2             2                   2   
      C6                      3             3                   3   
      D4                      2             2                   2   
7     A1                      1             1                   1   
      A15                     1             1                   1   
      A7                      2             2                   2   
      B2                      3             3                   3   
      B3                      1             1                   1   
      C11                     2             2                   2   
      C6                      1             1                   1   
      D14                     1             1                   1   
      D4                      4             4                   4   
      E18                     1             1                   1   
      E5                      1             1                   1   
8     A1                      8             8                   8   
      A15                     1             1                   1   
      A7                      1             1                   1   
      B2                     12            12                  12   
      B3                      6             6                   6   
      C11                     7             7                   7   
      C6                      9             9                   9   
      D14                     1             1                   1   
      D4                     11            11                  11   
      E13                     5             5                   5   
      E18                     6             6                   6   
      E5                      1             1                   1   
9     A1                     12            12                  12   
      A15                     2             2                   2   
      A7                      3             3                   3   
      B2                     15            15                  15   
      B3                     10            10                  10   
      C11                     8             8                   8   
      C6                      4             4                   4   
      D14                     6             6                   6   
      D4                      8             8                   8   
      E13                     4             4                   4   
      E18                     5             5                   5   
      E5                      1             1                   1   
10    A1                      1             1                   1   
11    A1                      1             1                   1   
      B3                      1             1                   1   
      D4                      1             1                   1   
12    A1      

In [8]:
# FREQUENCY
print(df.OFFENSE_CODE_GROUP.value_counts())


Drug Violation                     25
Other                              21
Motor Vehicle Accident Response    11
Larceny                             9
Robbery                             8
                                   ..
Warrant Arrests                     1
Service                             1
Auto Theft Recovery                 1
Landlord/Tenant Disputes            1
Burglary - No Property Taken        1
Name: OFFENSE_CODE_GROUP, Length: 66, dtype: int64


In [9]:
from scipy.stats import mode

def unique_nan(s):
    return s.nunique(dropna=False)
def count_nulls(s):
    return s.size - s.count()

agg_func_custom_count = {
    'OFFENSE_CODE_GROUP': ['count', 'nunique', 'size', unique_nan, count_nulls, set]
}
df.groupby(['DISTRICT']).agg(agg_func_custom_count)

OFFENSE_CODE_GROUP                                      \
                      count nunique size unique_nan count_nulls   
DISTRICT                                                          
A1                       28      19   28         19           0   
A15                       4       4    4          4           0   
A7                        6       6    6          6           0   
B2                       33      24   33         24           0   
B3                       19      12   19         12           0   
C11                      17      12   17         12           0   
C6                       18      11   18         11           0   
D14                      12      10   12         10           0   
D4                       27      16   27         16           0   
E13                      12      10   12         10           0   
E18                      14      12   14         12           0   
E5                        4       4    4          4           0   

                                                             
                                                        set  
DISTRICT                                                     
A1        {Biological Threat, Firearm Violations, Prison...  
A15       {Assembly or Gathering Violations, Arson, Medi...  
A7        {Other, Aircraft, Drug Violation, HOME INVASIO...  
B2        {Verbal Disputes, Firearm Violations, License ...  
B3        {Violations, Other, Property Lost, Fire Relate...  
C11       {Residential Burglary, Other, Larceny, Drug Vi...  
C6        {Commercial Burglary, Other, Larceny, Drug Vio...  
D14       {Violations, Other, Police Service Incidents, ...  
D4        {Commercial Burglary, Other, Bomb Hoax, Recove...  
E13       {Other, Police Service Incidents, Recovered St...  
E18       {Violations, Investigate Person, Landlord/Tena...  
E5        {Police Service Incidents, Motor Vehicle Accid...

In [10]:

df.drop_duplicates(subset="OFFENSE_CODE_GROUP", inplace=True)
df.OFFENSE_CODE_GROUP.tolist()

['Larceny',
 'Vandalism',
 'Towed',
 'Investigate Property',
 'Motor Vehicle Accident Response',
 'Auto Theft',
 'Verbal Disputes',
 'Robbery',
 'Fire Related Reports',
 'Other',
 'Property Lost',
 'Medical Assistance',
 'Assembly or Gathering Violations',
 'Larceny From Motor Vehicle',
 'Residential Burglary',
 'Simple Assault',
 'Violations',
 'Harassment',
 'Ballistics',
 'Property Found',
 'Police Service Incidents',
 'Drug Violation',
 'Missing Person Reported',
 'Investigate Person',
 'Fraud',
 'Aggravated Assault',
 'License Plate Related Incidents',
 'Other Burglary',
 'Warrant Arrests',
 'Disorderly Conduct',
 'Harbor Related Incidents',
 'Counterfeiting',
 'Liquor Violation',
 'Firearm Discovery',
 'Landlord/Tenant Disputes',
 'Auto Theft Recovery',
 'Service',
 'Operating Under the Influence',
 'Confidence Games',
 'Restraining Order Violations',
 'Firearm Violations',
 'Missing Person Located',
 'License Violation',
 'Commercial Burglary',
 'Search Warrants',
 'Recovered St